In [1]:
#!pip3 install "gym[atari]"
import gym
import sys
from PIL import Image, ImageDraw
import numpy as np
import pandas as pd
import neural_tape_controller
#Положительные числа - положительные награды.
import tasks 
import optimize
import pickle5 as pickle

In [2]:
# HIDE OUTPUT
#!wget http://www.atarimania.com/roms/Roms.rar 
#!unrar x -o+ ./Roms.rar >./roms
!python3 -m atari_py.import_roms ./ROMS >./roms

In [2]:
def query_environment(name):
    env = gym.make(name)
    spec = gym.spec(name)
    print(f"Action Space: {env.action_space}")
    print(f"Observation Space: {env.observation_space}")
    print(f"Max Episode Steps: {spec.max_episode_steps}")
    print(f"Nondeterministic: {spec.nondeterministic}")
    print(f"Reward Range: {env.reward_range}")
    print(f"Reward Threshold: {spec.reward_threshold}")
query_environment("ALE/Tetris-v5")

Action Space: Discrete(5)
Observation Space: Box(0, 255, (210, 160, 3), uint8)
Max Episode Steps: 27000
Nondeterministic: False
Reward Range: (-inf, inf)
Reward Threshold: None


A.L.E: Arcade Learning Environment (version 0.7.5+db37282)
[Powered by Stella]


In [3]:
def encode_simple(X):
  #как сделать простое кодирование? Ну давай разобьём картинку двумя разными сетками и измерим яркость
  #на входе одиночная картинка
  y_lst = []
  sz = np.shape(X)
  count_squares = 7
  for i in range(count_squares):
    for j in range(count_squares):
      x = X[int(i*sz[0]/count_squares):int((i+1)*sz[0]/count_squares), int(j*sz[1]/count_squares):int((j+1)*sz[1]/count_squares)]
      y_lst.append(np.nanmean(x))

  count_squares = 3
  for i in range(count_squares):
    for j in range(count_squares):
      x = X[int(i*sz[0]/count_squares):int((i+1)*sz[0]/count_squares), int(j*sz[1]/count_squares):int((j+1)*sz[1]/count_squares)]
      y_lst.append(np.nanmean(x))

  count_squares = 2
  for i in range(count_squares):
    for j in range(count_squares):
      x = X[int(i*sz[0]/count_squares):int((i+1)*sz[0]/count_squares), int(j*sz[1]/count_squares):int((j+1)*sz[1]/count_squares)]
      y_lst.append(np.nanmean(x))
  #размер: 62
  return np.array(y_lst)

In [4]:
n_actions = 5
embed_size = 62#это эмбеддинг картинки
nt = neural_tape_controller.nt_controller(input_size=embed_size,output_size=n_actions)
try:
    with open(root_dir+f'./genoms/best_genom_tetris.pkl', 'rb') as f:
        genom = pickle.load(f)
        genom = genom[-1]
    print('loaded successfully')
except Exception:
    genom = nt.nn.disassemble_genom()

In [4]:
n_actions = 5
embed_size = 62#это эмбеддинг картинки
nt = neural_tape_controller.nt_controller(tacts=1,input_size=embed_size,output_size=n_actions)
genom = nt.nn.disassemble_genom()

In [5]:
def test_atary(genom, draw=False,game_name="ALE/Tetris-v5",seed=0):

  env = gym.make(game_name)
  env.reset()
  n_actions = env.action_space.n
  state_dim = env.observation_space.shape#здесь картинка
  #print('n_actions',n_actions,'state_dim',state_dim)
  #1/0
  state_dim = embed_size#это эмбеддинг картинки
  controller = neural_tape_controller.nt_controller(tacts=1,genom=np.array(genom),input_size=state_dim,output_size=n_actions) 
  out_tape = np.zeros(30)
  reward_sum = 0

  #seed=1
  np.random.seed(seed)
  env.seed(seed)
  while True:
      action = np.ravel(out_tape)
      action += np.random.rand(len(action))*np.std(action)*0.05#игра детерминистическая, иначе рандома не будет
      #print('action',int(np.argmax(action)),action)
      #исполнить env
      state, reward, done,_ = env.step(int(np.argmax(action)))
      t=pd.Timestamp.now()
      if done: 
        reward_sum += -1
        break
      #state = encode_mobnet(np.array([state]))
      state = encode_simple(state)
      reward_sum += (reward + 0.001)
      if np.shape(state)[0]>1:
          shp = np.shape(state)
          state = np.reshape(state,[1,shp[0]])
      out_tape = controller.act(state,reward,done)
      if draw:
        globals()['video'].append(Image.fromarray(env.render("rgb_array")))
  return reward_sum

In [6]:
def multy_test(genom,draw=False):
  if draw:
    globals()['video'] = []

  game_name = "ALE/Tetris-v5"
  #один энвайронмент, разные сиды
  q_arr = []
  for seed in range(3):
    q_arr.append(test_atary(genom, draw=draw,game_name=game_name,seed=seed))
  return np.sum(q_arr)/len(q_arr)

In [7]:
%%time
q=test_atary(genom,game_name="ALE/Tetris-v5", draw=False)
print('q=',q)

q= -0.5829999999999997
CPU times: user 5.27 s, sys: 15.4 s, total: 20.7 s
Wall time: 2 s


In [7]:
%%time
q=multy_test(genom)
print('q=',q)
#q=multy_test(genom)
#print('q=',q)

q= -0.5829999999999997
CPU times: user 16 s, sys: 48.3 s, total: 1min 4s
Wall time: 6 s


In [9]:
%%time
q=test_atary(genom)
print('q=',q)

q= -0.5829999999999997
CPU times: user 5.07 s, sys: 15 s, total: 20 s
Wall time: 1.88 s


In [10]:
print(pd.Timestamp.now())

2022-05-23 15:37:35.771122


In [8]:
print(1)
opt = optimize.optimizer(multy_test, genom_size=len(genom),parallel_cores=12)

1


In [9]:
try:
    with open('genoms/best_genom_tetris.pkl', 'rb') as f:
        opt.best_genoms = pickle.load(f)
    print('loaded successfully')
except Exception:
    pass

loaded successfully


In [10]:

opt.function(opt.best_genoms[-1])

for i in range(1000):
    print('opt#',i)
    opt.optimize()
    with open('genoms/best_genom_tetris.pkl', 'wb') as f:
        pickle.dump(opt.best_genoms,f,protocol=pickle.HIGHEST_PROTOCOL)
        print('WRITTEN')

opt# 0
random trial
scores for optimizers augmented [1.e+10 1.e+10 1.e+10 1.e+10 1.e+10 1.e+10 1.e+10 1.e+10]
chosen evol_soft previous_result: nan per tacts: nan
iteration 0 y= [ 0.545 -0.023 -0.023]
iteration 1 y= [ 0.545  0.009 -0.023]


KeyboardInterrupt: 

In [11]:
with open('./genoms/best_genom_tetris.pkl', 'rb') as f:
        genom = pickle.load(f)
        genom = genom[-1]

In [12]:
q=multy_test(genom,draw=True)
print('q=',q)

q= 0.5449999999999579


In [13]:
video[0].save(
    './out_videos/tetris.gif',
    save_all=True,
    append_images=video[1:], 
    optimize=True,
    duration=100,
    loop=0
)

In [16]:
#noise stategy
#with open('genoms/best_genom_tetris.pkl', 'rb') as f:
    #genom = pickle.load(f)[-1]
globals()['video'] = []
noise = np.random.normal(size=[len(genom)])*1
for i in range(1):
    genom = genom+noise
    q=test_atary(genom, draw=True,game_name="ALE/Tetris-v5",seed=0)
    np.random.seed(i)
    noise = np.random.normal(size=[len(genom)])*1



In [17]:
video[0].save(
    './out_videos/tetris.gif',
    save_all=True,
    append_images=video[1:], 
    optimize=True,
    duration=10,
    loop=0
)

In [15]:
with open('genoms/best_genom_tetris.pkl', 'wb') as f:
    pickle.dump(genom,f,protocol=pickle.HIGHEST_PROTOCOL)
    print('WRITTEN')

WRITTEN
